In [1]:
import sys
sys.path.append('/home/jiajunb/neural-dimension-reduction')

In [2]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F

from src.models.distance_modeling import SurveyorDataSet, Surveyor, thesis_kl_div_add_mse_loss, RetrieveSystem

import copy
from src.models.DenseNetwork import loss

torch.manual_seed(0)

In [3]:
device = torch.device('cuda:0')

In [4]:
best_model = Surveyor()
best_model.load_state_dict(torch.load('../saves/surveyor.on.full.100')['best_model'])
best_model.eval()

val_dataset = SurveyorDataSet.from_df('/home/jiajunb/neural-dimension-reduction/data/dev.csv')
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=1000, pin_memory=True)

gold = loss.nearest_neighbors(val_dataset.data.clone(), top_k=1, device='cuda')

In [5]:
best_model = best_model.to(device)

In [6]:
def extract_embeddings(data_loader, model):
    model.eval()
    embedding = list()
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            x = batch[0].to(device)
            out = model.encode_batch(x)
            embedding.append(out.cpu())
    return torch.cat(embedding, dim=0)

val_nn_loader = DataLoader(TensorDataset(val_dataset.data.clone()), batch_size=1000, pin_memory=True)
val_x_embedded = extract_embeddings(val_nn_loader, best_model)

In [7]:
retriever = RetrieveSystem(best_model)
block_list = torch.arange(val_x_embedded.shape[0])
cls_pred_nn_top, p_distances_nn_top = retriever.retrieve_corpus(val_x_embedded, block_list, val_x_embedded)


In [8]:
print(cls_pred_nn_top.shape)
print(p_distances_nn_top.shape)
print(gold[2].shape)

torch.Size([1000, 20])
torch.Size([1000, 20])
torch.Size([1000, 1])


In [13]:
cls_pred_nn_top[:10, 0]

tensor([707, 707,  40, 942,  40, 707,   3,  35, 834, 674])

In [15]:
gold[1].view(-1)[:10]

tensor([202, 598,  55, 309,  97, 390, 729, 601, 930, 672])

In [19]:
(cls_pred_nn_top[:, 0] == gold[1].view(-1, 1)).sum()

tensor(237)

In [21]:
len(gold[1])

1000

In [37]:
# recall @1
float((cls_pred_nn_top[:, 0] == gold[1].view(-1)).sum()) / len(gold[1])

0.002

In [38]:
cls_pred_nn_top[:10, :5]

tensor([[707, 129, 585,  30, 223],
        [707,  30, 129, 598, 585],
        [ 40, 301, 952, 751, 655],
        [942, 787, 678, 525, 945],
        [ 40, 301, 952, 751, 655],
        [707, 129, 585,  30, 598],
        [  3, 787, 942, 678, 525],
        [ 35, 613, 864, 683, 976],
        [834, 686, 973, 696,  13],
        [674, 794, 172, 361, 528]])

In [39]:
(cls_pred_nn_top[:, :5] == gold[1].reshape(-1, 1)).sum()

tensor(24)

In [41]:
# recall@5
float((cls_pred_nn_top[:, :5] == gold[1].view(-1, 1)).sum()) / len(gold[1])

0.024

In [42]:
# recall@10
float((cls_pred_nn_top[:, :10] == gold[1]).sum()) / len(gold[1])

0.045

In [43]:
# recall@20
float((cls_pred_nn_top[:, :20] == gold[1]).sum()) / len(gold[1])

0.073